### Installing InvokeAI - Required

In [ ]:
import os
import subprocess

env = os.environ.copy()

!pip install pillow==9.5.0 requests==2.31.0 xformers==0.0.20 triton==2.0.0
!pip install git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install git+https://github.com/Birch-san/k-diffusion.git@mps#egg=k-diffusion
!pip install git+https://github.com/invoke-ai/clipseg.git@relaxed-python-requirement#egg=clipseg
!pip install git+https://github.com/invoke-ai/PyPatchMatch@0.1.4#egg=pypatchmatch
!pip install 'InvokeAI[xformers]==3.6.0' --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu118
!pip install --force-reinstall numpy

!mamba install openssh -y 

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 
### Configuration and downloading default models - Required

In [ ]:
!mkdir /kaggle/temp/
!mkdir /kaggle/temp/invokeai
!mkdir /kaggle/temp/invokeai/configs

#@markdown Download only the default model in initial configuration.
#@markdown Checking this prevents running out of space in Colab.

defaultOnly = True #@param {type:"boolean"}
skipWeights = True #@param {type:"boolean"}
skipSupportModels = False #@param {type:"boolean"}
noFullPrecision = True

#@markdown This step usually takes about 2 minutes with only the default model and no weights.

#@markdown You can ignore "File exists" warnings in the output.

cmd = 'invokeai-configure --root_dir /kaggle/temp/invokeai --yes'

if defaultOnly:
  cmd += ' --default_only'

if skipWeights:
  cmd += ' --skip-sd-weights'

if skipSupportModels:
  cmd += ' --skip-support-models'

subprocess.run(cmd, shell=True, env=env)

import fileinput
import os
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

if noFullPrecision:
  model_install_file = find('model_install_backend.py', '/opt/conda/lib/')
  print('modifying file ' + str(model_install_file))
  for line in fileinput.input(model_install_file, inplace=True):
    if ('precision = torch_dtype(choose_torch_device())' in line):
       line = line.replace('torch_dtype(choose_torch_device())', 'torch.float16')
    print(line, end='')

#  Add the Dreamshaper SDXL model (optional)



In [ ]:
import os.path
from os import path

# Install the SDXL base model
def installSdxl(env):
  installCmd = 'invokeai-model-install --add "Lykon/dreamshaper-xl-1-0" --root_dir /kaggle/temp/invokeai'
  subprocess.run(installCmd, shell=True, env=env)
  
alreadyInstalled = True
sdxlBaseSubfolderName = ''
modelsPath = '/kaggle/working/stablemodels/'
sdxlBaseSubfolderName = '/dreamshaper-xl-1-0'
workSdxlMainFolder = modelsPath + 'sdxl/main'
if not path.exists(workSdxlMainFolder):
    os.makedirs(workSdxlMainFolder, exist_ok=True)
    alreadyInstalled = False

tempModelsSdxlFolder = '/kaggle/temp/invokeai/models/sdxl/'
tempSdxlMainFolder = tempModelsSdxlFolder + 'main'

subprocess.run('rm -rf ' + tempModelsSdxlFolder, shell=True, env=env)
if path.exists(tempModelsSdxlFolder):
    subprocess.run('rmdir ' + tempModelsSdxlFolder, shell=True, env=env)

if not alreadyInstalled:
    if not path.exists(tempModelsSdxlFolder):
      os.makedirs(tempModelsSdxlFolder, exist_ok=True)
    subprocess.run('ln -s '+workSdxlMainFolder+' '+tempModelsSdxlFolder, shell=True, env=env)
    installSdxl(env)
else:
    if not path.exists(tempSdxlMainFolder):
      os.makedirs(tempSdxlMainFolder, exist_ok=True)
    subprocess.run('ln -s '+workSdxlMainFolder + sdxlBaseSubfolderName+' '+ tempSdxlMainFolder, shell=True, env=env)
    updateModelsYaml = True
    with open('/kaggle/temp/invokeai/configs/models.yaml') as f:
      if 'dreamshaper-xl-1-0' in f.read():
        updateModelsYaml = False
    if updateModelsYaml:
      with open('/kaggle/temp/invokeai/configs/models.yaml', 'a') as file:
        lines = [
          'sdxl/main/dreamshaper-xl-1-0:\n',
          '  path: sdxl/main/dreamshaper-xl-1-0\n',
          '  description: Dreamshaper XL (12 GB)\n',
          '  variant: normal\n',
          '  format: diffusers\n'
        ]
        file.writelines(lines)

# Start Invokeai

In [ ]:
#Option 2: Starting the Web UI with RemoteMoe
port = 7860

!mkdir  ~/.ssh/
!touch  ~/.ssh/known_hosts
!ssh-keyscan -t rsa remote.moe >> ~/.ssh/known_hosts
!rm /root/.ssh/id_rsa
!ssh-keygen -t rsa -b 4096 -f /root/.ssh/id_rsa -q -N ""
!invokeai-web --root /kaggle/temp/invokeai/ & ssh -R 80:127.0.0.1:9090 -o StrictHostKeyChecking=no -i ~/.ssh/id_rsa remote.moe